In [2]:
import numpy as np
from meshparty import meshwork, trimesh_vtk

skel_dir = 'data'

ModuleNotFoundError: No module named 'meshparty'

The files in the directory `data` contain skeletons of all of the basket cells in a central column of primary visual cortex.
The number is the so-called `root id` of a given cell at a given state of proofreading, and the suffix `.h5` indicates that it's an "hdf5" file, which is a generic file format that can hold all sorts of data. In this case, these h5 files are formatted as a "meshwork" object, which holds information about the morphology and connectivity of neurons.

In [2]:
import glob

# Glob lets you do shell-like file lists in python
files = glob.glob(f"{skel_dir}/*.h5")

This command loads a neuron file into python

In [3]:
nrn = meshwork.load_meshwork(files[0])

Since we are interested in dendrite-dendrite interactions, the first thing we want to do is limit our analysis to dendrites.

To show you how this goes, we're going to plot the cell along the way. We've been using vtk as a quick way to plot cells in 3d.

The following two lines of code will bring up a 3d plot of the neuron, with the dendrite in red and the axon in blue.
You'll have to close the window to continue.

In [11]:
ska = trimesh_vtk.skeleton_actor(
    nrn.skeleton, line_width=2, vertex_data=nrn.anno.is_axon.skel_mask
)

trimesh_vtk.render_actors([ska])

setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x7fd2d660bc00) at 0x7fd2d850ff40>

You can see all of the vertices of the object like so:

In [12]:
nrn.skeleton.vertices

array([[509720., 456576., 829640.],
       [511968., 455232., 830640.],
       [510680., 456320., 829920.],
       ...,
       [869496., 407336., 990160.],
       [870608., 407680., 990440.],
       [682944., 453376., 864040.]])

How many vertices are there?

In [13]:
len(nrn.skeleton.vertices)

5607

But now we want to mask out the axon and leave only the dendrite.
When I made these skeletons to begin with, I already labeled the axon and dendrite (which is how we did the colors before), so we can just use that.
However, these objects contain both skeletons (which are trees) and "meshes", which are somewhat more fleshed out.
In this case, these aren't true meshes, but they still capture a bit more structure of the cells.

Meshwork objects have meshes, skeletons, and annotations ( like axon/dendrite labels or like synapses) that are all linked together into a core object.
One key operation is masking, which limits an object to a given subset.
The function `apply_mask` takes a boolean array on the mesh, which we can get similar to the colors above.


In [19]:
nrn.apply_mask(~nrn.anno.is_axon.mesh_mask)

After applying a mask, you can see the effect by plotting the object again.

In [20]:
ska = trimesh_vtk.skeleton_actor(
    nrn.skeleton, line_width=2, vertex_data=nrn.anno.is_axon.skel_mask.astype(int)
)

trimesh_vtk.render_actors([ska])

/Users/caseys/Work/code/core_tools/MeshParty/meshparty/trimesh_vtk.py:613: RuntimeWarning: invalid value encountered in true_divide
  data = data / np.nanmax(data)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x7fd2d4963000) at 0x7fd2b9bd72e0>

This time, you should see just a red part, and the skeleton will be much smaller.

You can reset the mask to the original state with `nrn.reset_mask()`

Now how many vertices are there?

In [21]:
len(nrn.skeleton.vertices)

1266

How do we measure the distance between dendrites?

As a first approximation, we can measure the closest distances between the dendrite vertices.

Scipy has an object called a K-d tree that does this very efficiently.
Let's use it here.

We're going to use the mesh vertices instead of the skeleton vertices to get a slightly richer representation.

In [22]:
nrn_other = meshwork.load_meshwork(files[1])
nrn_other.apply_mask(~nrn_other.anno.is_axon.mesh_mask)

In [23]:
from scipy import spatial

In [24]:
kdt = spatial.KDTree(nrn_other.mesh.vertices)
ds, inds = kdt.query(nrn.mesh.vertices) # This returns the distance and closest index in the tree object for each of the vertices queried.

What is the closest distance between the two dendrites?

In [27]:
np.min(ds)/1000   # The vertex units are in nanometers, so let's divide by 1000 to bring into microns

8.914678233116438

In [28]:
np.argmin(ds)

1517

In [32]:
nrn.seg_id

864691135988773120

In [31]:
nrn_other.seg_id

864691135946995809

In [30]:
nrn.mesh.vertices[np.argmin(ds)] / [4,4,40]

TrackedArray([170972., 115400.,  19954.])

Now, how would you go about getting this minimum distance for all pairs of cells?